In [1]:
import pandas as pd
import re
import sqlalchemy
import unicodedata
import difflib
import xlwings as xw
from datetime import datetime
from googletrans import Translator
from os import listdir
from os.path import isfile, join

In [2]:
def get_full_cost(x, full_cost_columns):
    cost = x[['Brutto', 'ZUS pracodawcy', 'ZUS Pracodawca', 'Skladki FGSP', 'Składki FGŚP', 'Skladki FP']].sum()
    cost = cost - x[[c for c in full_cost_columns if 'Zas' in c]].sum()

    return cost

def get_protected_sheet_data(PATH):
    wb = xw.Book(PATH)

    salaries = []
    for sheet in wb.sheets:
        print(sheet)

        data_from_sheet = sheet.range('A1').expand().options(pd.DataFrame, index=False, header=True).value

        data_from_sheet['Etat'] = data_from_sheet.apply(lambda x: x['Wymiar etatu L']/x['Wymiar etatu M'], axis=1)

        data_from_sheet.rename(columns={"Tytul": "Stanowisko",
                                        "Kwota": "Stawka",
                                        "kwota": "Stawka",
                                        "Wartość": "Stawka",
                                        "wartość": "Stawka",
                                        "ETAT": 'Etat'}, inplace=True)

        data_from_sheet['FinanceSheet'] = sheet.name

        data_from_sheet['Date'] = data_from_sheet['Data']
        full_cost_columns = data_from_sheet.columns.values
        data_from_sheet['EnovaEmployerCost'] = data_from_sheet.apply(lambda x: get_full_cost(x, full_cost_columns), axis=1)

        salaries.append(data_from_sheet[['Kod', 'Pracownik', 'Stanowisko', 'Etat',
                                         'Typ umowy', 'FinanceSheet', 'Date', 'Stawka', 'EnovaEmployerCost']])

    return salaries

def etat_to_number(x):
    g = re.match('([0-9])/([0-9])', str(x).replace("'", ""))
    if g is None:
        return g
    g = g.groups()
    if len(g) == 1:
        return g[0]
    return float(g[0])/float(g[1])

def get_name(x):
    x = unicodedata.normalize('NFD', x.lower()).replace('ł', 'l').encode('ascii', 'ignore').decode()
    if '(' in x:
        full_name = re.match('(.{1,}) (.{1,}) \(.{1,}\)', x).groups()
    else:
        full_name = re.match('(.{1,}) (.{1,})', x).groups()
    return full_name[1].title() + ' ' + full_name[0].title()

In [3]:
mypath = 'C:\\Users\\anicieja\\Documents\OBJ\\PL_automation\\source_new\\total'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
onlyfiles = [file for file in onlyfiles if not '~' in file]
onlyfiles

['Total_payroll_OBSS_01-09.2018.xlsx']

In [5]:
salaries_array = []
for name in onlyfiles:
    print(name)
    salaries_array.append(pd.concat(get_protected_sheet_data('C:\\Users\\anicieja\\Documents\\OBJ\\PL_automation\\source_new\\total\\' + name)))

Total_payroll_OBSS_01-09.2018.xlsx


<Sheet [Total_payroll_OBSS_01-09.2018.xlsx]total_OBSS_UOP>


In [12]:
salaries = pd.concat(salaries_array)

for col in salaries.columns:
    print(col)
# salaries.head()

ValueError: No objects to concatenate